# Understanding Bayesian Networks
### Part B: Inference in Bayesian Network
In this notebook we make inference in a Credit Card Fraud Detection System (a Bayesian Network) using following two methods:
1. Variable Elimination
2. Gibbs Sampling

In [72]:
# Importing the libraries
import random
import copy
import numpy as np

In [73]:
# Define the Bayesian network structure as a dictionary
# Each key is a random variable and value is a list of parent nodes
network = {
    "Travel": [],
    "Foreign Purchase": ["Travel"],
    "Owns Device": [],
    "Online Purchase": ["Owns Device"],
    "Fraud": ["Travel", "Online Purchase"]
}

# Define the conditional probability tables
# Make sure the order should match the order of parents as defined in the network
cpt_tables = {
    "Travel": [[0.05, 0.95]],
    "Foreign Purchase": [[0.88, 0.12],      # Travel = 1
                         [0.0001, 0.9999]], # Travel = 0
    "Owns Device": [[0.7, 0.3]],
    "Online Purchase": [[0.4, 0.6],         # Owns Device = 1
                        [0.05, 0.95]],      # Owns Device = 0
    "Fraud": [[0.995, 0.005],               # Travel = 1, Online Purchase = 1
              [0.85, 0.15],                 # Travel = 1, Online Purchase = 0
              [0.8, 0.2],                   # Travel = 0, Online Purchase = 1
              [0.75, 0.25]]                 # Travel = 0, Online Purchase = 0
}

#### Variable Elimination Method

In [74]:
# Funtion to straighten the CPT table, i.e., convert a 2D list to a 1D list and reverse it
def straighten_cpt_table(cpt_table):
    result = []
    for x in cpt_table:
        for y in x:
            result.append(y)
    # Reverse the list
    return result[::-1]

In [75]:
def Initialize_Factors(network, cpt_tables):
    # Initialize factors as an empty list
    factors = []
    
    # Iterate over the network
    for var, parents in network.items():
        # Variables in the factor are the parents plus the current variable
        vars = parents.copy()
        vars.append(var)
        # Table is the CPT table of the variable
        factor = {
            "Variables": vars,
            "Table": straighten_cpt_table(cpt_tables[var])
        }
        
        # Append the factor to the list of factors
        factors.append(factor)
    
    # Return the list of initialized factors
    return factors

In [76]:
# Funtion to perform Pointwise multiplication of two factors
def Multiply_Factors(factor1, factor2):
    # Create a new factor with variables from both factors
    new_variables = copy.deepcopy(factor1["Variables"])
    for var in factor2["Variables"]:
        if var not in new_variables:
            new_variables.append(var)
    
    # Find common variables between the two factors
    common_vars = []
    for var in factor1["Variables"]:
        if var in factor2["Variables"]:
            # Store the index of the common variable in both factors
            common_vars.append({"Name": var, "Index1": factor1["Variables"].index(var), "Index2": factor2["Variables"].index(var)})
    
    # New table to store the result of pointwise multiplication
    new_table = []
    # Iterate over all possible values of the variables in the new factor
    for i in range(len(factor1["Table"])):
        # Find the values of the common variables in factor1
        common_vars_values_1 = []
        for var in common_vars:
            if i & (2 ** (len(factor1["Variables"])- 1 - var["Index1"])):
                common_vars_values_1.append(True)
            else:
                common_vars_values_1.append(False)
        
        # Iterate over all possible values of the variables in the new factor
        for j in range(len(factor2["Table"])):
            # Find the values of the common variables in factor2
            common_vars_values_2 = []
            for var in common_vars:
                if j & (2 ** (len(factor2["Variables"])- 1 - var["Index2"])):
                    common_vars_values_2.append(True)
                else:
                    common_vars_values_2.append(False)
            
            # If for current values of common variables, the values in both factors are not same, skip to next iteration
            common_vars_match = True
            for k in range(len(common_vars)):
                if common_vars_values_1[k] != common_vars_values_2[k]:
                    # Values of common variables do not match
                    common_vars_match = False
                    break
            
            # Skip if not match
            if not common_vars_match:
                continue
            
            # If common variables match, multiply the values of the two factors and add to the new table
            new_table.append(factor1["Table"][i] * factor2["Table"][j])
            
    # Create the result factor with new variables and new table
    result_factor = {
        "Variables": new_variables,
        "Table": new_table
    }
    
    # Return the result factor
    return result_factor

In [77]:
# Funtion to perform sum operation on a factor to eliminate a variable
def Sum_Out(factor, variable):
    # New variables will be all variables in the factor except the variable to be eliminated
    new_variables = copy.deepcopy(factor["Variables"])
    new_variables.remove(variable)
    # Find the index of the variable to be eliminated
    index = factor["Variables"].index(variable)
    
    # New table to store the result of sum operation
    new_table = []
    # Iterate over all possible values of the variables in the new factor
    for i in range(len(factor["Table"])):
        # If the current value of the variable to be eliminated is True, we already took care of it in the previous iteration
        if i & (2 ** (len(factor["Variables"]) - 1 - index)):
            continue
        
        # If the current value of the variable to be eliminated is False, find the corresponding value where the variable is True
        set_i = i + (2 ** (len(factor["Variables"]) - 1 - index))
        
        # Add the values of the two corresponding entries in the table and add to the new table
        new_table.append(factor["Table"][i] + factor["Table"][set_i])
    
    # Create the result factor with new variables and new table
    result_factor = {
        "Variables": new_variables,
        "Table": new_table
    }
    
    # Return the result factor
    return result_factor

In [78]:
# Function to calculate the probability using Variable Elimination
def Calculate_Probability_Variable_Elimination(network, cpt_tables, query={}, evidence={}):
    # Get the list of query, evidence and hidden variables (variables not in query or evidence)
    query_var = list(query.keys())
    evidence_var = list(evidence.keys())
    hidden_var = list(set([var for var in network.keys()]) - set(query_var) - set(evidence_var))
    
    # Initialize factors
    factors = Initialize_Factors(network, cpt_tables)
    
    # Update factors that are observed in the evidence by setting the values of the variables in the evidence
    for var in evidence_var:
        # Find the factor that contains the variable
        for factor in factors:
            # If the variable is not in the factor, skip to the next factor
            if var not in factor["Variables"]:
                continue
            
            # Find the index of the variable in the factor
            index = factor["Variables"].index(var)
            # New table to store the updated values of the factor
            new_table = []
            
            # Iterate over factor table
            for i in range(len(factor["Table"])):
                # If the value of the variable is True for current iteration
                # and the value of the variable in evidence is True, keep the entry
                if evidence[var] and i & (2 ** (len(factor["Variables"]) - 1 - index)):
                    new_table.append(factor["Table"][i])
                
                # Else if the value of the variable is False for current iteration
                # and the value of the variable in evidence is False, keep the entry
                elif not evidence[var] and not i & (2 ** (len(factor["Variables"]) - 1 - index)):
                    new_table.append(factor["Table"][i])
                
                # Else, skip the entry
            
            # Update the table of the factor and remove the variable from the variables list
            factor["Variables"].remove(var)
            factor["Table"] = new_table
    
    # Eliminate hidden variables one by one
    for var in hidden_var:
        # To store the factors that contain the variable to be eliminated
        curr_factors = []
        # To store the factors that do not contain the variable to be eliminated
        new_factors = []
        
        # Find the factors that contain the variable to be eliminated and store them in curr_factors list
        for factor in factors:
            if var in factor["Variables"]:
                curr_factors.append(factor.copy())
            # If the variable is not in the factor, keep the factor as it is
            else:
                new_factors.append(factor.copy())
        
        # If there are no factors that contain the variable to be eliminated, skip to the next variable
        if len(curr_factors) == 0:
            continue
        
        # Multiply all factors that contain the variable to be eliminated one by one
        new_factor = curr_factors[0]
        if len(curr_factors) > 1:
            # Multiply all factors that contain the variable to be eliminated
            for factor in curr_factors[1:]:
                new_factor = Multiply_Factors(new_factor, factor)
        
        # Sum out the variable to be eliminated in the new factor
        new_factor = Sum_Out(new_factor, var)
        
        # Add the new factor to the list of factors for next iteration
        new_factors.append(new_factor.copy())
        factors = new_factors.copy()
    
    # Now finally, we have 2 or less factors left
    final_factor = factors[0]
    # Multiply the factors if there are more than one factor left
    if len(factors) > 1:
        for factor in factors[1:]:
            final_factor = Multiply_Factors(final_factor, factor)
    
    if len(query_var) != 1:
        return "Invalid Query"
    
    # Result of the query variable
    result = {
        True: final_factor["Table"][1],
        False: final_factor["Table"][0]
    }
    
    # Normalize the final probabilities
    result[True] /= result[True] + result[False]
    result[False] /= result[True] + result[False]
    
    # Return the result of the query variable
    return round(result[query[query_var[0]]], 5)

In [79]:
# Answering queries using Variable Elimination
# a. P(Fraud = True)
print("a. Prior Probability of Fraud transaction: ", Calculate_Probability_Variable_Elimination(network, cpt_tables, query={"Fraud": True}, evidence={})) 

# b. P(Fraud = True | Owns Device = True)
print("b. Probability of Fraud transaction given customer owns a device: ", Calculate_Probability_Variable_Elimination(network, cpt_tables, query={"Fraud": True}, evidence={"Owns Device": True})) 

# c. P(Fraud = True | Travel = True)
print("c. Probability of Fraud transaction given customer is travelling: ", Calculate_Probability_Variable_Elimination(network, cpt_tables, query={"Fraud": True}, evidence={"Travel": True})) 

a. Prior Probability of Fraud transaction:  0.77115
b. Probability of Fraud transaction given customer owns a device:  0.7769
c. Probability of Fraud transaction given customer is travelling:  0.89277


#### Gibbs Sampling Method

In [80]:
# Function to perform generate samples using Gibbs Sampling
def Generate_Gibbs_Sample(network, cpt_tables, evidence, num_samples=10000, burn_in=1000):
    # Initialize the samples list
    samples = []
    
    # Get the variables that are not in the evidence
    non_evidence = [var for var in cpt_tables.keys() if var not in evidence.keys()]
    
    # Initialize a sample with random values
    sample_i = {var: bool(np.random.randint(2)) for var in cpt_tables.keys()}
    
    # Update the sample with the values from the evidence
    for var, val in evidence.items():
        sample_i[var] = val
        
    # Generate samples
    for i in range(num_samples + burn_in):
        # Randomly select a variable that is not in the evidence
        var = random.choice(non_evidence)
        
        # Get the parents of the variable
        parents = network[var]
        
        # Find the index of the conditional probability table for the variable based on the values of the parents
        index = 0
        for j in range(len(parents)):
            # If the parent value is True, then add 2^(k-1-j) to the index, where k is the number of parents
            if sample_i[parents[j]]:
                index += (2 ** (len(parents) - j - 1))
        
        # If the variable is False, then the index is the last index of the conditional probability table
        # Subtract the index from the total number of rows in the conditional probability table to get the correct index
        index = len(cpt_tables[var]) - 1 - index
        
        # Update the value of the variable in the sample based on the conditional probability table by using probability as weights
        sample_i[var] = random.choices([True, False], weights=[cpt_tables[var][index][0], cpt_tables[var][index][1]])[0]
        
        # Ignore the samples during the burn-in period to stabilize the distribution
        if i >= burn_in:
            samples.append(copy.deepcopy(sample_i))
            
    return samples

In [81]:
# Function to calculate the probability of query variable given the evidence using Gibbs Sampling and generated samples
def Calculate_Probability_Gibbs_Sampling(network, cpt_tables, query={}, evidence={}, num_samples=100000, burn_in=1000):
    num_correct_samples_with_required_query = 0
    num_correct_samples_with_all_evidence_correct = 0
    
    # Generate samples using Gibbs Sampling
    samples = Generate_Gibbs_Sample(network, cpt_tables, evidence, num_samples, burn_in)
    
    # Check if the samples satisfy the evidence and query variables
    for sample in samples:
        all_evidence_var_correct = True
        all_query_var_correct = True
        
        # Check if all evidence variables are correct
        for var, val in evidence.items():
            if sample[var] != val:
                all_evidence_var_correct = False
                break
            
        # Check if all query variables are correct
        for var, val in query.items():
            if sample[var] != val:
                all_query_var_correct = False
                break
        
        # If all evidence variables are correct, increment the count of correct samples (i.e., denominator of the probability)
        if all_evidence_var_correct:
            num_correct_samples_with_all_evidence_correct += 1
            # If all query variables are correct, increment the count of correct samples (i.e., numerator of the probability)
            if all_query_var_correct:
                num_correct_samples_with_required_query += 1
    
    return num_correct_samples_with_required_query / num_correct_samples_with_all_evidence_correct

In [82]:
# Answering queries using Gibbs Sampling
# a. P(Fraud = True)
print("a. Prior Probability of Fraud transaction: ", Calculate_Probability_Gibbs_Sampling(network, cpt_tables, query={"Fraud": True}, evidence={})) 

# b. P(Fraud = True | Owns Device = True)
print("b. Probability of Fraud transaction given customer owns a device: ", Calculate_Probability_Gibbs_Sampling(network, cpt_tables, query={"Fraud": True}, evidence={"Owns Device": True})) 

# c. P(Fraud = True | Travel = True)
print("c. Probability of Fraud transaction given customer is travelling: ", Calculate_Probability_Gibbs_Sampling(network, cpt_tables, query={"Fraud": True}, evidence={"Travel": True})) 

a. Prior Probability of Fraud transaction:  0.77146
b. Probability of Fraud transaction given customer owns a device:  0.78086
c. Probability of Fraud transaction given customer is travelling:  0.89213
